# AIAA Data Profiling Notebook

This notebook connects to a Snowflake database, queries and processes data, performs minimal exploration, and logs segmented profiles to WhyLabs.

## Importing Libraries

In [1]:
import pandas as pd
import snowflake.connector
import whylogs as why
from datetime import datetime, timezone
from whylogs.core.schema import DatasetSchema
from whylogs.core.segmentation_partition import ColumnMapperFunction, SegmentationPartition
from whylogs.api.writer.whylabs import WhyLabsWriter
import os
import numpy as np

## Ingesting Data

### Define Snowflake Connection

Make sure you write down your password before running the next cell

In [2]:
USER_NAME = "n-ITD_03968_WARRANTY_ANALYTICS_SVC"
PASSWORD = ""  # Enter your password
ACCOUNT_IDENTIFIER = "paccar"
WAREHOUSE = "ITD_REN_03_968_WARRANTY_AI_LARGE_WH"
DATABASE = "OMEGA_PROD_DB"
SCHEMA = "ITD_AACOE_WARRANTY_AI"

conn = snowflake.connector.connect(
    user=USER_NAME,
    password=PASSWORD,
    account=ACCOUNT_IDENTIFIER,
    warehouse=WAREHOUSE,
    database=DATABASE,
    schema=SCHEMA,
)

### Define Features

In [4]:
relevant_features = [
    "total_labor_hours_claimed",
    "total_labor_cost_claimed",
    "total_parts_cost_claimed",
    "total_misc_cost_claimed",
    "causal_part_model_no",  # high cardinality
    "chassisdivcd",
    "claim_category_cd",
    "engine_hours_quantity",
    "failure_location_cd",  # high cardinality
    "failure_type_cd",
    "supplier_cd",
    "vehicle_mileage_quantity",
]
relevant_features = ", ".join([f'"{column}"' for column in relevant_features])

### Defining and executing query

In [5]:
query = f"""
SELECT 
    {relevant_features},
    l."real_auto_approve_flag" AS cls_output, 
    l."real_auto_approve_score" AS score_output,
    CASE WHEN w.TOTAL_REQUESTED_AMT = w.TOTAL_APPROVED_AMT THEN TRUE ELSE FALSE END AS GROUND_TRUTH,
    l."quality_control_flag",
    w.CLAIMDIVCD AS division,
    w.CLAIM_RESOLUTION_DTTM
FROM "OMEGA_PROD_DB"."ITD_AACOE_WARRANTY_AI"."ML_CLAIMS_LOG" l
INNER JOIN "OMEGA_PROD_DB"."ITD_NGW_REPORTING"."VW_PRW_DEALER_WARRANTY_CLAIM_ALL" w
    ON l."dealer_claim_id" = w.DEALER_CLAIM_ID
    AND l."vin17" = w.VIN17
WHERE TRY_TO_DATE(l."claim_submit_dt") >= '2024-06-01'
ORDER BY l."claim_submit_dt";
"""

In [6]:
cur = conn.cursor()
cur.execute(query)
df = cur.fetch_pandas_all()

The following cell will close the connection to Snowflake. It is a good practice to do this, but not strictly necessary. 

In [7]:
cur.close()
conn.close()

The following cell does some DataFrame cleaning and filtering before we make profiles out of it.

In [8]:
pd.set_option("display.max_columns", None)
df.columns = df.columns.str.lower()
df[["cls_output", "ground_truth", "quality_control_flag"]] = df[["cls_output", "ground_truth", "quality_control_flag"]].astype(int)

cutoff_date = pd.to_datetime("2024-07-01").date()
monitored_claims = df[df["claim_resolution_dttm"].dt.date >= cutoff_date]

monitored_claims.head()

,total_labor_hours_claimed,total_labor_cost_claimed,total_parts_cost_claimed,total_misc_cost_claimed,causal_part_model_no,chassisdivcd,claim_category_cd,engine_hours_quantity,failure_location_cd,failure_type_cd,supplier_cd,vehicle_mileage_quantity,cls_output,score_output,ground_truth,quality_control_flag,division,claim_resolution_dttm
2,6.00,1295.400,586.73,647.70,None,P,ENG,3350.0,044004001,51,78584,110991.0,0,0.559740,0,0,P,2024-07-09 06:55:53.292
3,0.50,30.000,0.00,3.00,None,F,TRK,NaN,002015035,39,None,16.0,1,0.958509,1,0,F,2024-07-10 11:00:14.388
4,1.40,284.578,267.21,142.28,None,P,ENG,7419.0,044002002,64,87164,281032.0,0,0.679893,1,0,P,2024-07-22 13:48:57.400
6,1.00,190.000,604.38,0.00,None,P,TRK,NaN,032002001,38,20542AB,306196.0,0,0.720525,1,0,P,2024-07-31 06:56:50.095
10,2.82,507.600,1636.85,0.00,None,P,ENG,10535.0,043004107,82,04383AB,311345.0,0,0.609506,0,0,P,2024-07-08 17:36:42.896


## Drift Injection Functions

### Drift for numerical features

In [9]:
SUDDEN_DRIFT_DAY_NUMBER = 24
SUDDEN_DRIFT_STD_DEVIATION_NUMBER = 5

GRADUAL_DRIFT_STD_DEVIATION_NUMBER = 5
GRADUAL_DRIFT_WINDOW_SIZE = 6

INCREMENTAL_DRIFT_START_DAY = 7
INCREMENTAL_DRIFT_END_DAY = 44
INCREMENTAL_DRIFT_MAX_STD_DEVIATION_NUMBER = 5

REOCURRING_CONCEPT_START_DAY = 7
REOCURRING_CONCEPT_END_DAY = 44
REOCURRING_CONCEPT_STD_DEVIATION_NUMBER = 5


def add_noise(
    df: pd.DataFrame,
    column: str,
    percentage: float = 0.65,
    added_std: float = None,
    value: float = None,
):
    data_to_add_noise = df.sample(frac=percentage)
    if added_std:
        std = data_to_add_noise[column].std()
        value = added_std * std
    data_to_add_noise[column] = data_to_add_noise[column] + value
    df.update(data_to_add_noise)

    return df


def add_null(
    df: pd.DataFrame,
    column: str,
    percentage: float = 0.07,
):
    data_to_add_noise = df.sample(frac=percentage)
    data_to_add_noise[column] = None
    df.update(data_to_add_noise)

    return df


def sudden_drift(df: pd.DataFrame, column: str, day_number: int):
    if day_number > SUDDEN_DRIFT_DAY_NUMBER:
        df = add_noise(df, column, added_std=SUDDEN_DRIFT_STD_DEVIATION_NUMBER)
    return df


def gradual_drift(df: pd.DataFrame, column: str, day_number: int):
    bin_number = int(day_number / GRADUAL_DRIFT_WINDOW_SIZE)
    offset_number = int(day_number % GRADUAL_DRIFT_WINDOW_SIZE)
    if offset_number >= GRADUAL_DRIFT_WINDOW_SIZE - bin_number:
        df = add_noise(df, column, added_std=GRADUAL_DRIFT_STD_DEVIATION_NUMBER)
    return df


def incremental_drift(df: pd.DataFrame, column: str, day_number: int):
    if day_number < INCREMENTAL_DRIFT_START_DAY:
        return df
    elif day_number > INCREMENTAL_DRIFT_END_DAY:
        added_std = INCREMENTAL_DRIFT_MAX_STD_DEVIATION_NUMBER
    else:
        added_std = (
            INCREMENTAL_DRIFT_MAX_STD_DEVIATION_NUMBER
            * day_number
            / (INCREMENTAL_DRIFT_END_DAY - INCREMENTAL_DRIFT_START_DAY)
        )
    df = add_noise(df, column, added_std=added_std)
    return df


def reocurring_concepts(df: pd.DataFrame, column: str, day_number: int):
    if (
        day_number < REOCURRING_CONCEPT_START_DAY
        or day_number > REOCURRING_CONCEPT_END_DAY
    ):
        return df
    else:
        return add_noise(df, column, added_std=REOCURRING_CONCEPT_STD_DEVIATION_NUMBER)

### Drift for categorical features

In [10]:
SUDDEN_DRIFT_CAT_DAY_NUMBER = 24
SUDDEN_DRIFT_CAT_TARGET = "K"  # Placeholder: Replace with your target category
SUDDEN_DRIFT_CAT_INCREASE_PROB = 0.6  # Suggestion: Adjust based on desired impact

GRADUAL_DRIFT_CAT_START_DAY = 20
GRADUAL_DRIFT_CAT_END_DAY = 60
GRADUAL_DRIFT_CAT_TARGET = "ENG"  # Placeholder: Replace with your target category
GRADUAL_DRIFT_CAT_MAX_INCREASE_PROB = 0.6  # Suggestion: Adjust based on desired impact

INCREMENTAL_DRIFT_CAT_START_DAY = 20
INCREMENTAL_DRIFT_CAT_END_DAY = 60
INCREMENTAL_DRIFT_CAT_TRANSITIONS = {
    "51": (
        "38",
        0.5,
    ),  # Placeholder: Define original-target pairs and max probabilities
}

REOCURRING_CONCEPT_CAT_CYCLE_LENGTH = (
    7  # Suggestion: Adjust based on the frequency of concept changes
)
REOCURRING_CONCEPT_CAT_DISTRIBUTIONS = {
    0: {
        "01674AA": 0.7,
        "15310AA": 0.3,
    },  # Placeholder: Define distributions for different cycles
    1: {"01674AA": 0.3, "15310AA": 0.7},
}

def sudden_drift_categorical(df, column, day_number):
    if day_number > SUDDEN_DRIFT_CAT_DAY_NUMBER:
        df[column] = df[column].apply(
            lambda x: np.random.choice(
                [SUDDEN_DRIFT_CAT_TARGET, x],
                p=[SUDDEN_DRIFT_CAT_INCREASE_PROB, 1 - SUDDEN_DRIFT_CAT_INCREASE_PROB],
            )
        )
    return df


def gradual_drift_categorical(df, column, day_number):
    if GRADUAL_DRIFT_CAT_START_DAY <= day_number <= GRADUAL_DRIFT_CAT_END_DAY:
        progress = (day_number - GRADUAL_DRIFT_CAT_START_DAY) / (
            GRADUAL_DRIFT_CAT_END_DAY - GRADUAL_DRIFT_CAT_START_DAY
        )
        increased_prob = GRADUAL_DRIFT_CAT_MAX_INCREASE_PROB * progress
        df[column] = df[column].apply(
            lambda x: np.random.choice(
                [GRADUAL_DRIFT_CAT_TARGET, x], p=[increased_prob, 1 - increased_prob]
            )
        )
    return df


def incremental_drift_categorical(df, column, day_number):
    if INCREMENTAL_DRIFT_CAT_START_DAY <= day_number <= INCREMENTAL_DRIFT_CAT_END_DAY:
        progress = (day_number - INCREMENTAL_DRIFT_CAT_START_DAY) / (
            INCREMENTAL_DRIFT_CAT_END_DAY - INCREMENTAL_DRIFT_CAT_START_DAY
        )
        for original, (target, max_prob) in INCREMENTAL_DRIFT_CAT_TRANSITIONS.items():
            transition_probability = progress * max_prob
            df[column] = df[column].apply(
                lambda x: np.random.choice(
                    [target if x == original else x, x],
                    p=[transition_probability, 1 - transition_probability],
                )
            )
    return df


def reoccurring_concepts_categorical(df, column, day_number):
    current_phase = day_number % REOCURRING_CONCEPT_CAT_CYCLE_LENGTH
    if current_phase in REOCURRING_CONCEPT_CAT_DISTRIBUTIONS:
        for category, prob in REOCURRING_CONCEPT_CAT_DISTRIBUTIONS[
            current_phase
        ].items():
            df[column] = df[column].apply(
                lambda x: np.random.choice(
                    [category if x == category else x, x], p=[prob, 1 - prob]
                )
            )
    return df

## Profiling and logging into Whylabs

### Setting up environment variables

Before you run the next cell, make sure you go to Whylabs and create a new resource. To do this go to the main Organization Dashboard and click on `Set up resource` 

Then write down the ID of the new resource and run the cell. Make sure you don't use another ID as you might be overwriting another project.

In [11]:
os.environ["WHYLABS_DEFAULT_ORG_ID"] = "org-d2HtfF"
os.environ["WHYLABS_API_KEY"] = (
    "M7f2mt6akM.6EXkWFgQhMgbDWAp0rSy9kVZ3FgFGf64sJJ1koegn8qAAkPtAHiJD:org-d2HtfF"
)
os.environ["WHYLABS_DEFAULT_DATASET_ID"] = "" # Your resource ID

### Define the segmentation to be done

In [12]:
segmentation_partition = SegmentationPartition(
    name="division,quality_control_flag",
    mapper=ColumnMapperFunction(col_names=["division", "quality_control_flag"]),
)

### Create profiles and log them into Whylabs

Note that the following cell is injecting drift to the data before creating the profiles. You can `comment those lines` of code to upload clean profiles.

In [13]:
# Ensure 'claim_submit_dt' is in datetime format
monitored_claims["claim_resolution_dttm"] = pd.to_datetime(monitored_claims["claim_resolution_dttm"])

# Group DataFrame by the 'claim_submit_dt' column
grouped = monitored_claims.groupby(monitored_claims["claim_resolution_dttm"].dt.date)

# Iterate over each group and log the data
for day_num, (date, group) in enumerate(grouped):
    # Set the timestamp for the profile
    timestamp = datetime.combine(date, datetime.min.time(), tzinfo=timezone.utc)

    # Inject drift into numerical features (comment this to upload profiles without injected drift)
    group = sudden_drift(group, "total_labor_hours_claimed", day_num)
    group = gradual_drift(group, "total_labor_cost_claimed", day_num)
    group = incremental_drift(group, "total_parts_cost_claimed", day_num)
    group = reocurring_concepts(group, "total_misc_cost_claimed", day_num)
    group = sudden_drift_categorical(group, "chassisdivcd", day_num)
    group = gradual_drift_categorical(group, "claim_category_cd", day_num)
    group = incremental_drift_categorical(group, "failure_type_cd", day_num)
    group = reoccurring_concepts_categorical(group, "supplier_cd", day_num)

    # Create profiles, outputs, and segments
    results = why.log_classification_metrics(
        group,
        target_column="ground_truth",
        prediction_column="cls_output",
        score_column="score_output",
        log_full_data=True,
        schema=DatasetSchema(
            segments={segmentation_partition.name: segmentation_partition}
        ),
        dataset_timestamp=timestamp,
    )
    # Upload results to whylabs
    results.writer("whylabs").write()

    # Print confirmation
    print(f"Logged profile for {date}")

/var/folders/9v/5kz9l0m96vx0p_crl5tbb1lh0000gp/T/ipykernel_72323/609856399.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  monitored_claims["claim_resolution_dttm"] = pd.to_datetime(monitored_claims["claim_resolution_dttm"])


⚠️ Initializing default session because no session was found.
Initializing session with config /Users/fquagliotti/Library/Application Support/whylogs/config.ini

✅ Using session type: LOCAL. Profiles won't be uploaded or written anywhere automatically.
Logged profile for 2024-07-01
Logged profile for 2024-07-02
Logged profile for 2024-07-03
Logged profile for 2024-07-04
Logged profile for 2024-07-05
Logged profile for 2024-07-06
Logged profile for 2024-07-07
Logged profile for 2024-07-08
Logged profile for 2024-07-09
Logged profile for 2024-07-10
Logged profile for 2024-07-11
Logged profile for 2024-07-12
Logged profile for 2024-07-13
Logged profile for 2024-07-14
Logged profile for 2024-07-15
Logged profile for 2024-07-16
Logged profile for 2024-07-17
Logged profile for 2024-07-18
Logged profile for 2024-07-19
Logged profile for 2024-07-20
Logged profile for 2024-07-21
Logged profile for 2024-07-22
Logged profile for 2024-07-23
Logged profile for 2024-07-24
Logged profile for 2024-07-

### Create a Reference Profile

In [14]:
ref_claims = df[df["claim_resolution_dttm"].dt.month == 6]
ref_profile = why.log(ref_claims).profile()
writer = WhyLabsWriter().option(reference_profile_name="reference_june")
writer.write(file=ref_profile.view())

(True, 'ref-VSzGu08p0NlOGbMa')

### Deleting a profile

In [15]:
# from datetime import datetime


# def datetime_to_timestamp(dt):
#     epoch = datetime.utcfromtimestamp(0)
#     return int((dt - epoch).total_seconds() * 1000)


# # convert '09/09/2024, 00:00:00' to a unix timestamp
# datetime_to_timestamp(datetime(2024, 9, 10, 0, 0, 0))

In [16]:
# ! curl -I -X 'DELETE' \
#   'https://api.whylabsapp.com/v0/organizations/org-d2HtfF/dataset-profiles/models/model-7?profile_start_timestamp=1725926400000&profile_end_timestamp=1725926400000' \
#   -H 'accept: application/json' \
#   -H 'X-API-Key: M7f2mt6akM.6EXkWFgQhMgbDWAp0rSy9kVZ3FgFGf64sJJ1koegn8qAAkPtAHiJD:org-d2HtfF'